In [3]:
import os, re

from math import ceil, floor, comb

In [4]:
def cascade(n, i):
    a = []
    for k in range(i, 1, -1):
        m = 0
        while comb(m, k) < n:
            m += 1
        a.append(m - 1)
        n -= comb(m - 1, k)
    a.append(n)
    a.append(0)
    a.reverse()
    return a


def kruskal_katona(n, i, r):
    a = cascade(n, i)
    s = 1
    for k in range(i, r, -1):
        s += comb(a[k], k - r)
    return s

In [5]:
def partitions(n, m, minimum=1):
    """
    Generator of partitions of n into m parts in lexicographic order with all parts at least minimum.
    Modification of Algorithm H from Knuth. http://www.cs.utsa.edu/~wagner/knuth/fasc3b.pdf
    Precondition: n >= m * minimum.
    """
    if m == 0:
        if n == 0:
            yield []
    elif m == 1:
        yield [n]
    elif m == 2:
        for i in range(minimum, floor(n / 2) + 1):
            yield [n - i, i]
    else:
        a = [n - (m - 1) * minimum]
        for i in range(m - 1):
            a.append(minimum)
        while True:
            yield a
            while a[1] < a[0] - 1:
                a[0] = a[0] - 1
                a[1] = a[1] + 1
                yield a
            j = 2
            s = a[0] + a[1] - 1
            while j < m and a[j] >= a[0] - 1:
                s += a[j]
                j += 1
            if j == m:
                return
            x = a[j] + 1
            a[j] = x
            j -= 1
            for i in range(j, 0, -1):
                a[i] = x
                s -= x
            a[0] = s

Lower bound of $e(K(n,k)\setminus S, S)$ for a given partition $p$ of the $|K(n,k)\setminus S|$ vertices into $c$ components.

In [6]:
def l(f, p):
    s = 0
    for x in p:
        s += f[x]
    return s

Search function of partitions for some $K(n,k)$ and a value of $c$. $G$, $d$, and $g$ denote the size, degree, and girth of $K(n,k)$ respectively. $a$ and $b$ are the number of singletons and $K_2$'s in $K(n,k)\setminus S$. The algorithm is described in Section 6.2.

In [7]:
def search(n, k, f, c):
    G = comb(n, k)
    d = comb(n - k, k)
    g = min(1 + 2 * ceil(k / (n - 2 * k)), 6 if n == 2 * k + 1 else 4)
    for S in range(c + 1, floor((n / k - 1) * c) + 1):
        u = min(d * S, floor((d + comb(n - k - 1, k - 1)) * S * (comb(n, k) - S) / comb(n, k))) # upper bound
        a_low = ceil((2 * c * (d - 1) - u) / (d - 2))
        a_high = 1
        while a_high <= c and kruskal_katona(a_high, n - k, n - 2 * k) <= S:
            a_high += 1
        for a in range(a_low, a_high):
            b_low = max(0, ceil((g * c - G + S + (1 - g) * a) / (g - 2)))
            b_high = 1
            while a + b_high <= c and a + 2 * b_high <= G - S and kruskal_katona(a + b_high, n - k, n - 2 * k) - b_high <= S:
                b_high += 1
            for b in range(b_low, b_high):
                for p in partitions(G - S - a - 2 * b, c - a - b, g):
                    if l(f, p) + f[2] * b + f[1] * a <= u:
                        print(p + [2] * b + [1] * a)

$K(9,4)$

From Appendix 6.1, for $6\leq |\mathcal{C}|\leq 16$, we need to find lower bounds on $e(\mathcal{C},S)$ in order to define $f_3$. Note the cases of $1\leq |\mathcal{C}|\leq 5$ are already covered in Appendix 6.1.

The nauty executables should be in your PATH environment variable.

In [ ]:
f = [None, 5, 8, None, None, None]
for c in range(6, 17):
    f.append(max(int(e) for e in re.findall(r'\d+ graphs : e=(\d+)', os.popen('geng -CtfD5 ' + str(c) + ' | countg -g6: --e').read())))
for c in range(17, 116):
    f.append(ceil(2 * c * (126 - c) / 126))

Alternatively, if nauty is not installed, initialize $f_3$ with hard coded values.

In [7]:
f = [None, 5, 8, None, None, None, 18, 21, 22, 25, 26, 27, 28, 29, 28, 31, 32]
for c in range(17, 116):
    f.append(ceil(2 * c * (126 - c) / 126))

Partition algorithm described in Section 6.2.

In [8]:
for c in range(5, 54):
    search(9, 4, f, c)

$K(10,4)$

Find lower bounds on $e(\mathcal{C},S)$.

In [ ]:
f = [None, 15, 28, None]
for c in range(4, 9):
    f.append(max(int(e) for e in re.findall(r'>C \d+ graphs with (\d+) edges', os.popen('geng -Ctv ' + str(c)).read())))
for c in range(9, 95):
    f.append(ceil(9 * c * (210 - c) / 210))

Hard coded values of $f_2$.

In [9]:
f = [None, 15, 28, None, 63, 72, 81, 88]
for c in range(9, 95):
    f.append(ceil(9 * c * (210 - c) / 210))

Partition algorithm from Section 6.2.

In [10]:
for c in range(48, 76):
    search(10, 4, f, c)

$K(7,3)$

In [12]:
f = [None, 4, 6, None, None, None, 12, 14]
for c in range(8, 26):
    f.append(ceil(2 * c * (35 - c) / 35))

In [13]:
for c in range(5, 14):
    search(7, 3, f, c)